In [5]:
import ccxt
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, concatenate, Attention, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

def fetch_multitimeframe_data(symbol='BTC/USDT', exchange_name='binance'):
    """Получение мультитаймфреймных данных с биржи"""
    exchange = getattr(ccxt, exchange_name)()
    timeframes = {
        '1m': '1m',
        '5m': '5m', 
        '15m': '15m',
        '1h': '1h',
        '4h': '4h',
        '1d': '1d',
        '1w': '1w'
    }
    
    data = {}
    
    for tf in timeframes:
        # Определение глубины исторических данных
        since = exchange.parse8601((datetime.now() - timedelta(days=365)).isoformat())
        
        try:
            # Получение OHLCV данных
            ohlcv = exchange.fetch_ohlcv(symbol, timeframes[tf], since=since)
            df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df.set_index('timestamp', inplace=True)
            
            # Фильтрация и проверка данных
            df = df[~df.index.duplicated(keep='first')]
            df = df.sort_index()
            df = df.dropna()
            
            data[tf] = df
            
        except Exception as e:
            print(f"Error fetching {tf} data: {str(e)}")
            continue
            
        # Соблюдение лимитов запросов
        time.sleep(exchange.rateLimit / 1000)
    
    return data

# Обновленная функция создания датасетов
def create_multitimeframe_dataset(raw_data, lookback_periods):
    datasets = {}
    scalers = {}
    
    for tf in TIMEFRAMES:
        if tf not in raw_data:
            continue
            
        # Нормализация данных
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(raw_data[tf][FEATURES])
        scalers[tf] = scaler
        
        # Создание окон
        X = []
        for i in range(lookback_periods[tf], len(scaled_data)):
            X.append(scaled_data[i-lookback_periods[tf]:i])
            
        datasets[tf] = np.array(X)
        
    return datasets, scalers

# Модифицированный блок main
if __name__ == "__main__":
    # Получение данных с биржи
    raw_data = fetch_multitimeframe_data()
    
    # Подготовка данных
    datasets, scalers = create_multitimeframe_dataset(raw_data, LOOKBACK_PERIODS)
    
    # Проверка доступности всех таймфреймов
    available_tfs = [tf for tf in TIMEFRAMES if tf in datasets]
    if not available_tfs:
        raise ValueError("No data available for any timeframe")
    
    # Создание и обучение модели
    model = create_model()
    
    # Подготовка данных для обучения
    X_train = {tf: data[:-100] for tf, data in datasets.items()}
    y_train = {
        'price_prediction': datasets['1m'][LOOKBACK_PERIODS['1m']:-100, -1, 3],
        'signal': (datasets['1m'][LOOKBACK_PERIODS['1m']+1:-99, 3] > 
                  datasets['1m'][LOOKBACK_PERIODS['1m']:-100, 3]).astype(int)
    }
    
    model.fit(
        X_train,
        y_train,
        epochs=50,
        batch_size=128,
        validation_split=0.2,
        verbose=1
    )
    
    # Пример прогнозирования
    current_data = {tf: data[-100:] for tf, data in datasets.items()}
    price_pred, signal_prob = model.predict(current_data)
    signal = generate_signal(
        current_price=raw_data['1m'].iloc[-1]['close'],
        predicted_price=price_pred[-1][0],
        threshold=0.002
    )
    print(f"Predicted signal: {signal}")

ImportError: Traceback (most recent call last):
  File "c:\Users\romep\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: Произошел сбой в программе инициализации библиотеки динамической компоновки (DLL).


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.